In [1]:
import os
import re
import glob

In [13]:
def get_values(dir):
    files = glob.glob(os.path.join(dir, '*.log'))
    sarss = []
    sars = None
    for filename in files:
        with open(filename, 'r') as f:
            lines = f.readlines()
        for i in range(len(lines)):
            _l = lines[i]
            if 'iteration' in _l:
                sars = None
            if 'Frequency: ' in _l:
                f = re.findall(r"[-+]?\d*\.\d+|\d+", _l)
                if len(f) == 10:
                    # ok
                    sars = dict(zip(['Frequency', 'Medium busy', 'Busy time', 'Active time'], f[6:]))
            elif sars is not None:
                # ok, got at least Frequency...
                if '] AP0 txpower:' in _l:
                    f = re.findall(r"[-+]?\d*\.\d+|\d+", _l)
                    if len(f) == 11:
                        sars.update(dict(zip(['txpower', 'new_txpower', 'channel', 'new_channel'], f[7:])))
                    if 'new Medium busy' in sars:
                        sarss.append(sars)
                    sars = None
                elif '] rewards:' in _l:
                    sars['r'] = _l.split('] rewards:')[1].replace('\n', '').strip()
                    if 'None' in sars['r'] :
                        sars = None  # error, skip
                    else:
                        try:
                            ## find new state
                            for j in range(1, 10):
                                _ll = lines[i + j]
                                if 'Frequency: ' in _ll:                                  
                                    f = re.findall(r"[-+]?\d*\.\d+|\d+", _ll)
                                    if len(f) == 10:
                                        # ok
                                        new_sars = dict(zip(['new Medium busy', 'new Busy time', 'new Active time'], f[7:]))
                                        sars.update(new_sars)
                                    break
                        except KeyError:
                            sars = None  # error skip
    return sarss

In [14]:
sarss = get_values('.')

In [15]:
print("Found", len(sarss))

Found 551


In [16]:
sarss

[{'Frequency': '2457',
  'Medium busy': '53.002860',
  'Busy time': '556',
  'Active time': '1049',
  'r': '4.972869',
  'new Medium busy': '51.528777',
  'new Busy time': '573',
  'new Active time': '1112',
  'txpower': '5.000000',
  'new_txpower': '2.000000',
  'channel': '10.000000',
  'new_channel': '11.000000'},
 {'Frequency': '2457',
  'Medium busy': '47.862454',
  'Busy time': '1030',
  'Active time': '2152',
  'r': '5.0',
  'new Medium busy': '48.119619',
  'new Busy time': '1062',
  'new Active time': '2207',
  'txpower': '2.000000',
  'new_txpower': '5.000000',
  'channel': '11.000000',
  'new_channel': '5.000000'},
 {'Frequency': '2457',
  'Medium busy': '47.119816',
  'Busy time': '2045',
  'Active time': '4340',
  'r': '3.0',
  'new Medium busy': '47.439108',
  'new Busy time': '2084',
  'new Active time': '4393',
  'txpower': '8.000000',
  'new_txpower': '8.000000',
  'channel': '2.000000',
  'new_channel': '5.000000'},
 {'Frequency': '2457',
  'Medium busy': '46.886514',